In [27]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
import torch
import cv2
#for calling one image
from PIL import Image

In [38]:
# Define a transform. Modify this based on your model's needs
transform = transforms.Compose([
    transforms.ToTensor(),               # Convert the PIL Image to a PyTorch tensor
    transforms.Resize((224, 224)),      # Resize to the size expected by your model (e.g., 224x224 for many pre-trained models)
    # transforms.Normalize(mean, std)   # If your model expects normalized data, add this line with the appropriate mean and std
])

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(transform(img))
            
    # Convert list of tensors to a single tensor (batch)
    images_tensor = torch.stack(images)
    return images_tensor


# Load images
folder_path = './dentalclassifier/Dentaldata/Gingivitis/Gingivitis'
gingivitus = load_images_from_folder(folder_path)

folder_path = './dentalclassifier/Dentaldata/hypodontia/hypodontia'
hypodontia = load_images_from_folder(folder_path)

folder_path = './dentalclassifier/Dentaldata/Datacaries/Datacaries/cariesoriginal/done'
caries = load_images_from_folder(folder_path)

folder_path = './dentalclassifier/Dentaldata/discoloration/discolorationoriginal/original'
discoloration = load_images_from_folder(folder_path)

folder_path = './dentalclassifier/Dentaldata/Calculus/Calculus'
calculus = load_images_from_folder(folder_path)

print(gingivitus.shape, hypodontia.shape, discoloration.shape, caries.shape, calculus.shape)  # Expected shape: [num_images, 3, 224, 224] assuming 3-channel (RGB) images resized to 224x224


torch.Size([2349, 3, 224, 224]) torch.Size([1251, 3, 224, 224]) torch.Size([183, 3, 224, 224]) torch.Size([219, 3, 224, 224]) torch.Size([1296, 3, 224, 224])


In [8]:
# combine tensors

# Create labels for the two datasets
gingivitis_labels = torch.tensor([0] * len(gingivitus))  # Label 0 for gingivitis
hypodontia_labels = torch.tensor([1] * len(hypodontia))  # Label 1 for hypodontia
discoloration_labels = torch.tensor([2] * len(discoloration))  # Label 2 for discoloration
caries_labels = torch.tensor([3] * len(caries))  # Label 3 for discoloration
calculus_labels = torch.tensor([4] * len(calculus   ))  # Label 4 for discoloration

# Create a new label for the group: 0 for gingivitis, 1 for hypodontia
group_labels = torch.cat((gingivitis_labels, hypodontia_labels, discoloration_labels, caries_labels, calculus_labels), dim=0)

# Combine the image tensors and labels into a single dataset
combined_images = torch.cat((gingivitus, hypodontia, discoloration, caries, calculus), dim=0)
dataset = TensorDataset(combined_images, group_labels)


In [20]:
from torch.utils.data import random_split

# create train and test set

# Define the sizes for your train, validation, and test sets (e.g., 70%, 15%, 15%)
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

# Split the dataset into train, validation, and test sets
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

# Create DataLoaders for each split
batch_size = 64  # Adjust as needed
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [36]:
# create model 
import torch.nn as nn
from tanden import SimpleCNN

# Initialize the model
num_classes = 5  # Five classes: gingivitis, hypodontia, discoloration, caries, calculus
model = SimpleCNN(num_classes)


# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
# Training and Validation Loop
num_epochs = 10  # Adjust as needed
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            
            # Calculate the loss
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Calculate training loss for the epoch
        train_loss = running_loss / len(train_loader)
        
        # Validation
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        # Calculate validation loss and accuracy for the epoch
        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * correct / total
        
        print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Acc: {val_accuracy:.2f}%")

# Example usage:
# Assuming you have 'train_dataloader' for training and 'val_dataloader' for validation
# train_model(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs=10)


In [23]:
# Test Loop
def test_model(model, test_loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # Calculate test loss and accuracy
    test_loss = test_loss / len(test_loader)
    test_accuracy = 100 * correct / total
    
    print(f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.2f}%")

# Example usage:
# Assuming you have 'test_dataloader' for testing
# test_model(model, test_dataloader)


In [24]:
# Train the model using the train_dataloader
train_model(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs=num_epochs)

# Test the model using the test_dataloader
test_model(model, test_dataloader)

Epoch [1/10] - Train Loss: 1.4788 - Val Loss: 0.9789 - Val Acc: 60.83%
Epoch [2/10] - Train Loss: 0.9498 - Val Loss: 0.8846 - Val Acc: 65.37%
Epoch [3/10] - Train Loss: 0.8116 - Val Loss: 0.8093 - Val Acc: 67.51%
Epoch [4/10] - Train Loss: 0.7044 - Val Loss: 0.6833 - Val Acc: 72.54%
Epoch [5/10] - Train Loss: 0.5908 - Val Loss: 0.6507 - Val Acc: 73.05%
Epoch [6/10] - Train Loss: 0.5157 - Val Loss: 0.7012 - Val Acc: 72.29%
Epoch [7/10] - Train Loss: 0.4267 - Val Loss: 0.7182 - Val Acc: 70.91%
Epoch [8/10] - Train Loss: 0.3817 - Val Loss: 0.7586 - Val Acc: 75.06%
Epoch [9/10] - Train Loss: 0.3273 - Val Loss: 0.7253 - Val Acc: 73.30%
Epoch [10/10] - Train Loss: 0.2940 - Val Loss: 0.6838 - Val Acc: 73.43%
Test Loss: 0.8045 - Test Accuracy: 72.49%


In [25]:
# Save the trained model checkpoint
checkpoint_path = "dental_classifier.pth"
torch.save(model.state_dict(), checkpoint_path)

In [31]:
# Load the trained model
# Uploading one image at the time and getting confidence interval

num_classes = 5  # Assuming 5 classes
model = SimpleCNN(num_classes)

# Load the model's weights (state_dict) from a saved checkpoint file
checkpoint_path = "dental_classifier.pth"  # Replace with your actual checkpoint file
model.load_state_dict(torch.load(checkpoint_path))
model.eval()  # Set the model to evaluation mode

# Define a transformation for preprocessing the single image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # Add other transformations as needed (e.g., normalization)
])

# Load and preprocess the single image you want to classify
image_path = "/Users/home_folder/Documents/Vakken/HTI/DBM140/dental project/Dentaldata/discoloration/discolorationoriginal/original/5.jpg"  # Replace with the path to your image
img = Image.open(image_path)
img = transform(img)
img = img.unsqueeze(0)  # Add a batch dimension (single image)

# Pass the image through the model
with torch.no_grad():
    output = model(img)

# Apply softmax to obtain class probabilities
softmax = nn.Softmax(dim=1)
probabilities = softmax(output)

# Interpret the model's prediction and confidence scores
class_labels = ["gingivitis", "hypodontia", "discoloration", "caries", "calculus"]
predicted_class = class_labels[torch.argmax(probabilities)]
confidence_score = torch.max(probabilities).item()

print(f"Predicted Class: {predicted_class}")
print(f"Confidence Score: {confidence_score:.2f}")

Predicted Class: discoloration
Confidence Score: 0.99
